<a href="https://colab.research.google.com/github/danil99152/prediction-of-government-tenders/blob/main/total_auction_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Changelog

#Import

In [2]:
!pip install catboost

In [3]:
from datetime import datetime, date
import math
import re
import gc

import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sbs
from statsmodels.graphics.mosaicplot import mosaic


from matplotlib.ticker import MultipleLocator
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import RMSprop
from keras.callbacks import Callback, TensorBoard, ModelCheckpoint
from keras.models import load_model

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf

import xgboost as xgb
import catboost as ctb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import joblib
import pickle

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
!gdown --id 1L6BffI4zO8ZMOtTOle3XABOXQ-5ZcfTU

Downloading...
From: https://drive.google.com/uc?id=1L6BffI4zO8ZMOtTOle3XABOXQ-5ZcfTU
To: /content/purchase_plans.csv
63.2MB [00:00, 174MB/s]


#Обработка данных

Считывание данных о заявках

In [5]:
contract_path = 'purchase_plans.csv'
dtypes = {'plan_number' : 'category', 
          'customer_legal_form' : 'category', 
          'positionNumber' : 'category', 
          'product_info' : 'category',
          'publicDiscussion' : 'category',
          'isEnergyServiceContract': 'category'}
data = pd.read_csv(contract_path, dtype = dtypes)

gc.collect()

0

1-КТРУ, 2-ОКПД2

In [6]:
ktru = data['product_info'].str.split(pat="-", expand=True)

In [7]:
ktru.columns = ['OKPD','KTRU']

In [8]:
data = pd.concat([data, ktru], axis=1)

In [9]:
data.dropna(inplace = True)

In [10]:
data = data.drop(['product_info'], axis=1)

In [11]:
data = data.reset_index(drop=True)

In [12]:
data.head()

,plan_number,customer_legal_form,positionNumber,positionCanceled,publicDiscussion,total,isEnergyServiceContract,time,OKPD,KTRU
0,201801133000253003,75404,201801133000253003000150,False,false,97522.23,false,44,95.12.10.000,00000001
1,201901133000089001,75404,201901133000089001000001,True,false,195000.00,false,31,35.11.10.000,00000001
2,201901133000074001,75404,201901133000074001000001,False,false,3117.87,false,365,19.20.21.125,00001
3,201901133000089001,75404,201901133000089001000016,False,false,16000.00,false,365,61.10.11.110,00000006
4,201901133000089001,75404,201901133000089001000001,True,false,195000.00,false,31,35.11.10.000,00000001


#Прогноз цены

In [42]:
cat_var = ['plan_number', 
          'customer_legal_form', 
          'positionNumber', 
          'publicDiscussion',
          'isEnergyServiceContract',
          'OKPD',
          'KTRU']

In [15]:
def errors(model, x_test, y_test):
  outputs = model.predict(x_test)
  error = np.absolute(outputs - y_test)
  mae = np.sum(error) / len(error)
  error = (outputs - y_test)**2
  mse = np.sum(error) / len(error)
  print("R2:", metrics.r2_score(outputs, y_test))
  print("MAE:", mae) #абсолютная ошибка (MAE)
  print("MSE:", mse) #среднеквадратичная ошибка (MSE)
  print("RMSE:", np.sqrt(mse)) #корень среднеквадратичной ошибки (RMSE)

In [41]:
x = data.drop(['positionCanceled', 'total'], axis=1)
y = data['total']

In [64]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle = False)

In [376]:
print(len(x_train))
print(len(y_train))

39363
39363


In [27]:
cat_features = cat_var
model_catboost = CatBoostRegressor(
    n_estimators = 1000,
    learning_rate = 0.01,
    max_depth = 16,
    task_type="GPU",
    devices='0:1',
).fit(x_train, y_train, cat_features,
      eval_set=(x_test, y_test),
             use_best_model=True,
             verbose=False)
errors(model_catboost, x_test, y_test)

R2: -16947.99576420564
MAE: 3534848.8727485323
MSE: 13993401652212.846
RMSE: 3740775.5415438716


In [65]:
enc = LabelEncoder()
for param in cat_var:
  x_train[param] = enc.fit_transform(x_train[param])
  x_test[param] = enc.fit_transform(x_test[param])
x_train.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,plan_number,customer_legal_form,positionNumber,publicDiscussion,isEnergyServiceContract,time,OKPD,KTRU
0,0,7,0,0,0,44,239,0
1,41,7,183,0,0,31,129,0
2,37,7,117,0,0,365,56,63
3,41,7,184,0,0,365,182,5
4,41,7,183,0,0,31,129,0


In [58]:
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

gc.collect()

88

In [45]:
rf_model = RandomForestRegressor(n_estimators=1000, 
                               bootstrap = True,
                               warm_start = True,
                               oob_score = True)
rf_model.fit(x_train, y_train.ravel())
errors(rf_model, x_test, y_test.ravel())

R2: -0.002165418418826004
MAE: 8007774.883023535
MSE: 2.3163623939143252e+16
RMSE: 152196005.00388718


In [46]:
model = XGBRegressor(seed=42,
                      n_estimators=1000,
                      max_depth=16,
                      learning_rate=0.01,
                      task_type="GPU",
                      devices='0:1',)
model.fit(x_train, y_train.ravel())
errors(model, x_test, y_test.ravel())

[21:41:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
R2: 0.004839528798751269
MAE: 2126063.3539188914
MSE: 621209147871603.1
RMSE: 24924067.642975196


In [47]:
#значимость признаков для леса
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
col = ['plan_number',	'customer_legal_form',	'positionNumber',	'publicDiscussion',	'isEnergyServiceContract',	'time',	'OKPD', 'KTRU']
ar_f=[]
for f, idx in enumerate(indices):
    ar_f.append([round(importances[idx],4), col[idx]])
print("Значимость признака:")
ar_f.sort(reverse=True)
ar_f

Значимость признака:


[[0.9071, 'positionNumber'],
 [0.0793, 'plan_number'],
 [0.0125, 'time'],
 [0.0009, 'OKPD'],
 [0.0002, 'customer_legal_form'],
 [1e-04, 'KTRU'],
 [0.0, 'publicDiscussion'],
 [0.0, 'isEnergyServiceContract']]

In [67]:
#стандартизация
x_train = preprocessing.scale(x_train)
x_test = preprocessing.scale(x_test)
y_train = preprocessing.scale(y_train)
y_test = preprocessing.scale(y_test)

In [68]:
lin_model = LinearRegression()
lin_model.fit(x_train, y_train)
errors(lin_model, x_test, y_test)

R2: -78.96768251434372
MAE: 0.3607978815460613
MSE: 1.0523618389442686
RMSE: 1.0258468886458


In [69]:
poly_model = LinearRegression()
polynomial_features= PolynomialFeatures(degree=3)
x_train_poly = polynomial_features.fit_transform(x_train)
x_test_poly = polynomial_features.fit_transform(x_test)
poly_model.fit(x_train_poly, y_train)
errors(poly_model, x_test_poly, y_test)

R2: -0.44679681547217287
MAE: 1.3870546261090382
MSE: 6.344609848330906
RMSE: 2.518850898392143
